# 01 Multiply 2D Matrice on the GPU

Cell A

In [16]:
import numpy as np
import numba
from numba import cuda, types

@cuda.jit
def mm(a, b, c):
    
    row, column = cuda.grid(2)
    sum = 0

    # Checking if row and column are within valid c boundary
    if row < c.shape[0] and column < c.shape[1]:
        
        # Computing partial products
        for k in range(a.shape[1]):
            sum += a[row, k] * b[k, column]
        # Defining the product of a and b
        c[row, column] = sum

(numba.pydata.org, n.d.)

## Steps taken:

- The first step was to prevent the row and column to be out of the boundary of the output matrix(c)
- As both matrices have the same shape, we could calculate the partial product iterating through the shape of a. The sum was incremented with the value of the partial products at each iteration.
- The value of sum was assigned to c.

Cell B (Pre-defined)

In [17]:
a = np.arange(16).reshape(4,4).astype(np.int32)
b = np.arange(16).reshape(4,4).astype(np.int32)
c = np.zeros_like(a)

d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_c = cuda.to_device(c)

grid = (2,2)
block = (2,2)
mm[grid, block](d_a, d_b, d_c)

Cell C (Pre-defined)

In [18]:
from numpy import testing
solution = a@b
output = d_c.copy_to_host()

testing.assert_array_equal(output, solution)

In [19]:
print(output)

[[ 56  62  68  74]
 [152 174 196 218]
 [248 286 324 362]
 [344 398 452 506]]


# 02 Add 2D Matrices Larger than the Grid Size

Cell A

In [25]:
@cuda.jit
def add_matrix_stride(A,B,C):
  # Unique thread index within the entire grid
  start = cuda.grid(1)
  # The total number of threads in the entire grid
  stride = cuda.gridsize(1)

  # Calculating sum for all elements
  for a in range(start, A.shape[0], stride):
      C[a] = A[a] + B[a]

Cell B (no modification needed)

In [23]:
A = np.arange(64*64).reshape(64*64).astype(np.int32)
B = A * 2
C = np.zeros_like(A)

d_A = cuda.to_device(A)
d_B = cuda.to_device(B)
d_C = cuda.to_device(C)

blocks = (6,6)
threads_per_block = (6,6)

add_matrix_stride[blocks, threads_per_block](d_A, d_B, d_C)

Cell C (for testing purposes only)

In [24]:
output = d_C.copy_to_host()
solution = A+B

testing.assert_array_equal(output, solution)

## Steps taken:

Although this task did not require lots of code, however probably due to some over complication I done great amount of research, however at the end I sticked to a very basic solution.
- The first two added lines were the cuda provided variables for calculating the unique thread indexes and the number of threads within the entire grid.
- The threads then will start to work through the data elements from the appropriate indexes until the data's boundaries with each iteration. 

# 03. Matrix Transposition using Shared Memory

Cell A (Matrix Transposition without Shared Memory)

In [9]:
@cuda.jit
def transpose(a_in, a_out):
  row = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
  col = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y

  a_out[row, col] = a_in[col, row]

Cell B (Matrix Transposition with Shared Memory)

In [10]:
import numba.types

@cuda.jit
def transpose_shared(a_in, a_out):
  # The block dimension is assumed (Tile size*Tile size - current task 32*32)
  # and also the multiple tile sized input.
  # At the assumed block dimension a padded size of tile dimension also has
  # been utilized.
  tile = cuda.shared.array((32, 33), numba.types.int32)

  # Thread a box indexing pattern using the tile dimension as stride
  x = cuda.blockIdx.x * 32 + cuda.threadIdx.x
  y = cuda.blockIdx.y * 32 + cuda.threadIdx.y
  
  # Transpose tile into shared memory
  for j in range(0, 32, 32):
      tile[cuda.threadIdx.y + j, cuda.threadIdx.x] = a_in[y + j, x] 

  # Wait for all threads in the block to finish updating shared memory
  cuda.syncthreads()  

  # Compute transposed offsets
  x = cuda.blockIdx.y * 32 + cuda.threadIdx.x
  y = cuda.blockIdx.x * 32 + cuda.threadIdx.y

  for j in range(0, 32, 32):
      a_out[y + j, x] = tile[cuda.threadIdx.x, cuda.threadIdx.y + j];

# Credits to Mark Harris

(Mark, 2013)

## Steps taken:

- A shared array was created (tile) with the assumed block dimension of 32x32. Also, the input is assumed here to be multiplication of the tile. The padded tile size was also utilized to define the block dimension what was the tile size plus one (32 + 1). Therefore, the shape of the given shared memory bank size was slanted, however this is to prevent bank conflict (Mark, 2013).
- The box indexing pattern was threaded by using the tile dimension as stride.
- The tile was transposed (copied) into the shared memory with the aid of a for loop.
- The threads were syncronized then by utilizing syncthreads(), so all threads in the block could finish updating the shared memory.
- The copied offsets then were computed.
- Finally, the data was transferred back from shared memory.

Cell C (Pre-defined)

In [11]:
size = 16384
a_in = cuda.to_device(np.arange(size*size, dtype=np.int32).reshape((size, size)))
a_out = cuda.device_array_like(a_in)

print(a_in.copy_to_host())

[[        0         1         2 ...     16381     16382     16383]
 [    16384     16385     16386 ...     32765     32766     32767]
 [    32768     32769     32770 ...     49149     49150     49151]
 ...
 [268386304 268386305 268386306 ... 268402685 268402686 268402687]
 [268402688 268402689 268402690 ... 268419069 268419070 268419071]
 [268419072 268419073 268419074 ... 268435453 268435454 268435455]]


Cell D (Pre-defined)

In [12]:
threads_per_block = (32, 32)
blocks_per_grid = (int(size/threads_per_block[0]), int(size/threads_per_block[1]))

Cell E (Pre-defined)

In [13]:
from numba.cuda.api import synchronize
%timeit transpose[blocks_per_grid, threads_per_block](a_in, a_out); cuda.synchronize()

The slowest run took 7.22 times longer than the fastest. This could mean that an intermediate result is being cached.
10 loops, best of 5: 26.1 ms per loop


Cell F (Pre-defined)

In [14]:
%timeit transpose_shared[blocks_per_grid, threads_per_block](a_in, a_out); cuda.synchronize()

The slowest run took 45.72 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 11.2 ms per loop


Cell G (Pre-defined)

In [15]:
print(a_out.copy_to_host())

[[        0     16384     32768 ... 268386304 268402688 268419072]
 [        1     16385     32769 ... 268386305 268402689 268419073]
 [        2     16386     32770 ... 268386306 268402690 268419074]
 ...
 [    16381     32765     49149 ... 268402685 268419069 268435453]
 [    16382     32766     49150 ... 268402686 268419070 268435454]
 [    16383     32767     49151 ... 268402687 268419071 268435455]]


# References

- Mark, H. (2013). An Efficient Matrix Transpose in CUDA C/C++ | NVIDIA Technical Blog. [online] Available at: https://developer.nvidia.com/blog/efficient-matrix-transpose-cuda-cc/
- numba.pydata.org. (n.d.). Examples — Numba 0.52.0.dev0+274.g626b40e-py3.7-linux-x86_64.egg documentation. [online] Available at: https://numba.pydata.org/numba-doc/dev/cuda/examples.html